# Day 27: 物体検出の基礎 - YOLOとSSDの実装

## Learning Objectives
- 物体検出の概念を理解する
- Two-stage detectors (Faster R-CNN) と One-stage detectors (YOLO, SSD) の違いを理解する
- YOLOとSSDのアーキテクチャと動作原理を学ぶ
- Anchor boxesとNMSの概念を理解する
- 実際の物体検出システムをPythonで実装する

---

# Part 1: 理論セクション（2時間）

## 1.1 物体検出とは？

物体検出（Object Detection）は、画像中の物体の位置とクラスを同時に行うタスクです。

**定義**:
- **物体の位置**: Bounding Box (バウンディングボックス) で表現
- **物体のクラス**: 「犬」「猫」「自動車」などのカテゴリ

**出力形式**:
```
 [x_min, y_min, x_max, y_max, confidence, class_id]
```

**例**: [10, 20, 110, 210, 0.95, 2]
- 位置: (10, 20)から(110, 210)の矩形領域
- 信頼度: 95%
- クラスID: 2（例: 自動車）

### 物体検出の応用例

1. **自動運転**: 歩行者、車、信号の検出
2. **監視カメラ**: 不審人物の追跡
3. **医療画像**: 病変部位の検出
4. **eコマース**: 商品の特定
5. **ロボット**: 物体の認識と操作

### 物体検出の評価指標

1. **IoU (Intersection over Union)**:
   - 予測とGround Truthの重なり度合い
   - 
   
   ```
   IoU = (予測 ∩ 正解) / (予測 ∪ 正解)
   ```

2. **mAP (mean Average Precision)**:
   - 各クラスのAPの平均
   - 物体検出で最も一般的な指標

## 1.2 Two-stage Detectors vs One-stage Detectors

### Two-stage Detectors（二段階検出器）

**代表例**: R-CNN → Fast R-CNN → Faster R-CNN

**特徴**:
- 1段階: Region Proposal（領域提案）
- 2段階: Classification and Bounding Box Regression
- **精度**: 高いが遅い
- **速度**: 約2-5 FPS

**Faster R-CNNのアーキテクチャ**:
```
入力画像 → ResNet → RPN → RoI Pooling → FC層 → クラス分類・回帰
```

**RPN (Region Proposal Network)**:
- 画像全体から物体の存在が可能性のある領域を提案
- Anchor boxesを使った提案生成

### One-stage Detectors（一段階検出器）

**代表例**: YOLO, SSD, RetinaNet

**特徴**:
- 1段階でClassificationとRegressionを行う
- **精度**: Two-stageより低いが高速
- **速度**: 15-60 FPS

**比較表**:
| 特性 | Two-stage | One-stage |
|------|-----------|----------|
| 精度 | 高い | 中程度 |
| 速度 | 遅い | 速い |
| 計算量 | 大 | 中 |
| 応用 | 高精度な検出 | リアルタイムアプリケーション |

## 1.3 YOLO (You Only Look Once)

### YOLOの基本思想

- 画像をグリッドに分割
- 各グリッドセルが物体の中心を検出
- 物体が存在するセルのみが予測を行う

**YOLO v3のアーキテクチャ**:
```
入力(416x416x3) → Darknet-53 → Feature Maps (3サイズ) → Yolo Layers
```

**Darknet-53**: ResNetを改良したネットワーク
- Residual blocksを使用
- 53層の深いネットワーク

### YOLOの予測仕組み

入力画像はS×Sのグリッドに分割されます。

**グリッドセルあたりの予測**:
- 5個のBounding Box
- 各Boxには:
  - 4つの座標 (x, y, w, h)
  - 1つの信頼度 (confidence)
  - 20個のクラス確率（COCOデータセットの場合）

**Boxの表現**:
```
[x, y, w, h, confidence, class_1, class_2, ..., class_20]
```

### YOLOの損失関数

```
L = λ_coord * L_coord + λ_obj * L_obj + λ_noobj * L_noobj + L_class
```

**各項目の意味**:
- L_coord: 座標の誤差 (x, y, w, h)
- L_obj: 物体がある場合の誤差
- L_noobj: 物体がない場合の誤差
- L_class: クラス分類の誤差

**重み係数**:
- λ_coord = 5
- λ_obj = 1
- λ_noobj = 0.5

座標誤差を重くする理由: 正確な位置が物体検出で最も重要

## 1.4 SSD (Single Shot MultiBox Detector)

### SSDの基本思想

- 複数のサイズのFeature Mapで検出
- 各Feature Mapに異なるサイズのAnchor boxesを適用
- Multi-scaleでの検出が可能

**SSD v3のアーキテクチャ**:
```
入力(300x300x3) → VGG16 → 特徴マルチスケール(6サイズ) → 検出ヘッド
```

**特徴マルチスケール**:
- 38x38 (小物体)
- 19x19
- 10x10
- 5x5
- 3x3
- 1x1 (大物体)

### SSDの予測仕組み

各Feature Map位置に複数のAnchor boxesを配置:

**検出ヘッドの出力**:
- 各Anchor boxに対して:
  - 4つの座標変換 (dx, dy, dw, dh)
  - 1つの物体信頼度
  - C個のクラス確率

**座標変換の式**:
```
cx = dx * pw + px
cy = dy * ph + py
w = pw * exp(dw)
h = ph * exp(dh)
```

- (px, py, pw, ph): Anchor boxの座標とサイズ
- (dx, dy, dw, dh): ネットワークからの予測値

## 1.5 Anchor Boxes

### Anchor boxesとは

- 事前に定義された矩形の集合
- 物体の多様な形状をカバーする
- 検出の性能向上に貢献

**Anchor boxの選定**:
- Aspect Ratios: 1:1, 1:2, 2:1
- Scales: 小・中・大

**YOLO v3のAnchor boxes**:
```
#[10,13],  [16,30],  [33,23],  # 小
#[30,61],  [62,45],  [59,119], # 中
#[116,90], [156,198], [373,326] # 大
```

### Anchor boxesの生成

```
 def generate_anchors(scales, aspect_ratios):
    """Anchor boxesを生成"""
    anchors = []
    for scale in scales:
        for ar in aspect_ratios:
            # w = scale * sqrt(ar), h = scale / sqrt(ar)
            w = scale * math.sqrt(ar)
            h = scale / math.sqrt(ar)
            anchors.append([w, h])
    return anchors
```

## 1.6 Non-Maximum Suppression (NMS)

### NMSの必要性

- 同じ物体に対する重複した検出を削除
- より良い予測を選択する

### NMSのアルゴリズム
```
1. すべての検出を信頼度でソート
2. 一番信頼度の高い検出を選択
3. その検出とIoUが閾値以上の検出をすべて削除
4. 2-3を繰り返し
```

### Soft NMS（改良版NMS）
```
score' = score × (1 - IoU)
```

## 1.7 画像処理のための数学的基礎

### IoUの計算

IoU（Intersection over Union）は二つの矩形領域の重なり度合いを測る指標です。

```
IoU = area(A ∩ B) / area(A ∪ B)
```

### 矩形の交差面積の計算

In [ ]:
def calculate_iou(box1, box2):
    """IoUを計算
    
    Args:
        box1: [x1, y1, x2, y2]
        box2: [x1, y1, x2, y2]
    
    Returns:
        IoU値
    """
    # 交差領域の座標を計算
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    
    # 交差面積を計算
    width_inter = max(0, x2_inter - x1_inter)
    height_inter = max(0, y2_inter - y1_inter)
    area_inter = width_inter * height_inter
    
    # 各矩形の面積を計算
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # 結合面積を計算
    area_union = area1 + area2 - area_inter
    
    # IoUを計算
    iou = area_inter / area_union if area_union > 0 else 0
    
    return iou

# テスト
box1 = [10, 10, 50, 50]  # 50x50の正方形
box2 = [20, 20, 60, 60]  # 重なる部分がある
box3 = [60, 60, 100, 100]  # 重なっていない

print(f"Box1: {box1}")
print(f"Box2: {box2}")
print(f"Box3: {box3}")
print(f"\nIoU(Box1, Box2) = {calculate_iou(box1, box2):.4f}")
print(f"IoU(Box1, Box3) = {calculate_iou(box1, box3):.4f}")

## 1.8 Tensor演算

物体検出ではテンソル（多次元配列）の演算が頻繁に行われます。

### ベクトルの内積と行列の乗算

In [ ]:
import math

def vector_add(v1, v2):
    """ベクトルの加法"""
    return [v1[i] + v2[i] for i in range(len(v1))]

def dot_product(v1, v2):
    """ベクトルの内積"""
    return sum(v1[i] * v2[i] for i in range(len(v1)))

def matrix_multiply(A, B):
    """行列の積"""
    rows_A, cols_A = len(A), len(A[0])
    rows_B, cols_B = len(B), len(B[0])
    
    if cols_A != rows_B:
        raise ValueError("行列のサイズが整合しません")
    
    C = [[0 for _ in range(cols_B)] for _ in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            C[i][j] = sum(A[i][k] * B[k][j] for k in range(cols_A))
    return C

# テスト
v1 = [1, 2, 3]
v2 = [4, 5, 6]
print(f"ベクトル v1 = {v1}")
print(f"ベクトル v2 = {v2}")
print(f"v1 + v2 = {vector_add(v1, v2)}")
print(f"v1 · v2 = {dot_product(v1, v2)}")

---

# Part 2: 実践セクション（2時間）

それでは、学んだ理論を実際に実装してみましょう！

## 2.1 環境準備

必要なライブラリをインポートします。

In [ ]:
# 必要なライブラリのインポート
import math
import random
import numpy as np  # 教育用のnumpyの一部機能を模倣
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# 乱数シードの固定
random.seed(42)
np.random.seed(42)

# 日本語フォントの設定
plt.rcParams['font.family'] = 'IPAexGothic'

## 2.2 Exercise 1: Simple Bounding Box Detector

まず、シンプルなバウンディングボックス検出器を実装してみましょう。

In [ ]:
class SimpleDetector:
    """シンプルな物体検出器"""
    
    def __init__(self, grid_size=7, num_classes=10):
        self.grid_size = grid_size
        self.num_classes = num_classes
        self.boxes_per_cell = 2
        
        # Anchor boxes（簡略化版）
        self.anchors = [
            [0.1, 0.1],  # 小物体
            [0.3, 0.3]   # 大物体
        ]
    
    def detect(self, image_features):
        """物体を検出
        
        Args:
            image_features: 画像の特徴マップ
            
        Returns:
            検出結果のリスト [x, y, w, h, confidence, class_id]
        """
        detections = []
        
        # 各グリッドセルを処理
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                # 特徴量に基づいて信頼度を計算（簡略化）
                confidence = min(1.0, abs(image_features[i, j]) / 0.5)
                
                if confidence > 0.3:  # 閾値処理
                    # 各anchor boxに対して予測
                    for anchor_idx, anchor in enumerate(self.anchors):
                        # 予測された位置とサイズ
                        cell_x = j + 0.5
                        cell_y = i + 0.5
                        
                        # 予測ボックスを計算
                        box = [
                            cell_x / self.grid_size,  # x（正規化）
                            cell_y / self.grid_size,  # y（正規化）
                            anchor[0],                # w
                            anchor[1],                # h
                            confidence,
                            0  # クラスID（簡略化）
                        ]
                        detections.append(box)
        
        return detections

# テスト用の特徴マップを生成
grid_size = 7
feature_map = np.zeros((grid_size, grid_size))

 # 物体のある場所に特徴量を設定
feature_map[2, 3] = 0.8  # 物体1
feature_map[4, 5] = 0.9  # 物体2
feature_map[1, 1] = 0.2  # ノイズ（検出されない）

# 検出器の作成と実行
detector = SimpleDetector(grid_size=7)
detections = detector.detect(feature_map)

print(f"検出結果: {len(detections)}個の物体を検出")
for i, det in enumerate(detections):
    print(f"物体{i+1}: 位置({det[0]:.2f}, {det[1]:.2f}), サイズ({det[2]:.2f}×{det[3]:.2f}), 信頼度={det[4]:.2f}")

## 2.3 Exercise 2: Anchor Box Generator

In [ ]:
def generate_anchors(base_sizes, aspect_ratios):
    """Anchor boxesを生成
    
    Args:
        base_sizes: 基本サイズのリスト
        aspect_ratios: アスペクト比のリスト
    
    Returns:
        Anchor boxesのリスト [w, h]
    """
    anchors = []
    for size in base_sizes:
        for ar in aspect_ratios:
            # w = size * sqrt(ar), h = size / sqrt(ar)
            w = size * math.sqrt(ar)
            h = size / math.sqrt(ar)
            anchors.append([w, h])
    return anchors

# YOLO v3風のAnchor boxes
scales = [10, 16, 33]  # スケール
aspect_ratios = [0.5, 1.0, 2.0]  # アスペクト比

anchors = generate_anchors(scales, aspect_ratios)

print(f"生成されたAnchor boxes ({len(anchors)}個):")
for i, anchor in enumerate(anchors):
    print(f"Anchor {i+1}: w={anchor[0]:.2f}, h={anchor[1]:.2f}")

## 2.4 Exercise 3: NMSの実装

In [ ]:
def non_max_suppression(boxes, scores, iou_threshold=0.5):
    """Non-Maximum Suppressionを実装
    
    Args:
        boxes: バウンディングボックスのリスト [x, y, w, h]
        scores: 各ボックスのスコア
        iou_threshold: IoUの閾値
    
    Returns:
        フィルタリングされたボックスのインデックスリスト
    """
    # スコアでソート
    indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    
    selected_indices = []
    
    while indices:
        # 最もスコアの高いボックスを選択
        current_index = indices[0]
        selected_indices.append(current_index)
        
        # 残りのボックスからIoUが高いものを削除
        remaining_indices = []
        for idx in indices[1:]:
            # 矩形を[cx, cy, w, h]から[x_min, y_min, x_max, y_max]に変換
            box1 = boxes[current_index]
            box1 = [box1[0] - box1[2]/2, box1[1] - box1[3]/2, 
                    box1[0] + box1[2]/2, box1[1] + box1[3]/2]
            
            box2 = boxes[idx]
            box2 = [box2[0] - box2[2]/2, box2[1] - box2[3]/2,
                    box2[0] + box2[2]/2, box2[1] + box2[3]/2]
            
            # IoUを計算
            iou = calculate_iou(box1, box2)
            
            if iou <= iou_threshold:
                remaining_indices.append(idx)
        
        indices = remaining_indices
    
    return selected_indices

# テストデータ
boxes = [
    [0.5, 0.5, 0.2, 0.2],  # 中心(0.5, 0.5), 幅0.2, 高さ0.2
    [0.5, 0.5, 0.25, 0.25],  # 重なるボックス
    [0.8, 0.8, 0.1, 0.1],  # 別の物体
    [0.7, 0.7, 0.15, 0.15]   # 重なるボックス
]
scores = [0.9, 0.85, 0.8, 0.75]

print("NMS前:")
for i, (box, score) in enumerate(zip(boxes, scores)):
    print(f"ボックス{i+1}: 位置({box[0]:.2f}, {box[1]:.2f}), サイズ({box[2]:.2f}×{box[3]:.2f}), スコア={score:.2f}")

# NMSの実行
selected_indices = non_max_suppression(boxes, scores, iou_threshold=0.4)

print("\nNMS後:")
for idx in selected_indices:
    print(f"選択されたボックス{idx+1}: スコア={scores[idx]:.2f}")

## 2.5 Exercise 4: YOLO Detectorの実装

In [ ]:
class YOLODetector:
    """YOLO風の物体検出器"""
    
    def __init__(self, grid_size=7, num_classes=20, num_anchors=2):
        self.grid_size = grid_size
        self.num_classes = num_classes
        self.num_anchors = num_anchors
        
        # Anchor boxesを定義
        self.anchors = [
            [0.1, 0.1],  # 小物体用
            [0.3, 0.3]   # 大物体用
        ]
    
    def forward(self, feature_maps):
        """Forward pass
        
        Args:
            feature_maps: 特徴マップ [grid_size, grid_size, anchors * (5 + num_classes)]
        
        Returns:
            前処理済みの予測
        """
        # シンプルな前処理
        predictions = []
        
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                for anchor_idx in range(self.num_anchors):
                    # 各anchor boxの予測を取得
                    base_idx = anchor_idx * (5 + self.num_classes)
                    
                    # シミュレーションとしてランダムな値を生成
                    # 実装ではネットワークの出力を使用
                    confidence = random.uniform(0, 1)
                    
                    # 座標とサイズを計算
                    cell_x = j + random.uniform(-0.5, 0.5)
                    cell_y = i + random.uniform(-0.5, 0.5)
                    
                    # 予測ボックス
                    box = [
                        cell_x / self.grid_size,  # x
                        cell_y / self.grid_size,  # y
                        self.anchors[anchor_idx][0],  # w
                        self.anchors[anchor_idx][1],  # h
                        confidence,  # confidence
                        0  # class_id（ランダム）
                    ]
                    
                    # クラス確率を生成
                    class_probs = [random.uniform(0, 1) for _ in range(self.num_classes)]
                    
                    predictions.append((box, class_probs))
        
        return predictions

    def filter_detections(self, predictions, confidence_threshold=0.5, iou_threshold=0.5):
        """予測をフィルタリング
        
        Args:
            predictions: forwardの出力
            confidence_threshold: 信頼度の閾値
            iou_threshold: NMSのIoU閾値
        
        Returns:
            フィルタリングされた検出結果
        """
        # 信頼度でフィルタリング
        filtered = []
        for box, class_probs in predictions:
            if box[4] > confidence_threshold:
                # 最も可能性の高いクラスを選択
                class_id = class_probs.index(max(class_probs))
                box[5] = class_id
                filtered.append(box)
        
        # NMSの適用
        if len(filtered) > 0:
            # データをNMS関数に適合させる
            boxes = [box[:4] for box in filtered]
            scores = [box[4] for box in filtered]
            
            selected_indices = non_max_suppression(boxes, scores, iou_threshold)
            
            final_detections = [filtered[i] for i in selected_indices]
            return final_detections
        
        return []

# YOLO検出器のテスト
detector = YOLODetector(grid_size=7, num_classes=20)

# シミュレーションの特徴マップ
feature_maps = np.random.rand(7, 7, 2 * (5 + 20))

# 予測の実行
predictions = detector.forward(feature_maps)
print(f"初期予測数: {len(predictions)}")

# フィルタリングの実行
detections = detector.filter_detections(predictions, confidence_threshold=0.7)
print(f"フィルタリング後の検出数: {len(detections)}")

print("\n検出結果:")
for i, det in enumerate(detections):
    print(f"物体{i+1}: 位置({det[0]:.2f}, {det[1]:.2f}), サイズ({det[2]:.2f}×{det[3]:.2f}),")
    print(f"          信頼度={det[4]:.2f}, クラスID={det[5]}")

## 2.6 Exercise 5: SSD Detectorの実装

In [ ]:
class SSDDetector:
    """SSD風の物体検出器"""
    
    def __init__(self, feature_sizes=[38, 19, 10, 5, 3, 1], num_classes=20):
        self.feature_sizes = feature_sizes
        self.num_classes = num_classes
        
        # 各特徴サイズのAnchor boxesを生成
        self.anchors_per_cell = 6
        self.anchors = self._generate_anchors()
    
    def _generate_anchors(self):
        """マルチスケールのAnchor boxesを生成"""
        all_anchors = []
        
        # 各特徴サイズに対して
        for idx, size in enumerate(self.feature_sizes):
            anchors_for_scale = []
            
            # セルあたりのAnchor boxes
            for i in range(self.anchors_per_cell):
                # スケールを特徴サイズに基づいて調整
                scale = 0.1 * (2 ** (idx / 3))
                aspect_ratio = [0.5, 1.0, 2.0][i % 3]
                
                w = scale * math.sqrt(aspect_ratio)
                h = scale / math.sqrt(aspect_ratio)
                
                anchors_for_scale.append([w, h])
            
            all_anchors.append(anchors_for_scale)
        
        return all_anchors

    def detect(self, feature_maps):
        """マルチスケールの物体検出
        
        Args:
            feature_maps: 各特徴サイズのマップのリスト
        
        Returns:
            すべての検出結果のリスト
        """
        all_detections = []
        
        for scale_idx, feature_map in enumerate(feature_maps):
            size = self.feature_sizes[scale_idx]
            anchors = self.anchors[scale_idx]
            
            # 各特徴マップを処理
            for i in range(size):
                for j in range(size):
                    for anchor_idx, anchor in enumerate(anchors):
                        # シミュレーションとしてランダムな値を生成
                        confidence = random.uniform(0, 1)
                        
                        # 位置変換をシミュレート
                        dx = random.uniform(-1, 1) * 0.2
                        dy = random.uniform(-1, 1) * 0.2
                        dw = random.uniform(-1, 1) * 0.2
                        dh = random.uniform(-1, 1) * 0.2
                        
                        # Anchor boxの中心とサイズ
                        px = (j + 0.5) / size
                        py = (i + 0.5) / size
                        pw = anchor[0]
                        ph = anchor[1]
                        
                        # 変換されたボックス
                        cx = dx * pw + px
                        cy = dy * ph + py
                        w = pw * math.exp(dw)
                        h = ph * math.exp(dh)
                        
                        # クラス確率
                        class_probs = [random.uniform(0, 1) for _ in range(self.num_classes)]
                        class_id = class_probs.index(max(class_probs))
                        
                        # 検出ボックス
                        detection = [cx, cy, w, h, confidence, class_id]
                        all_detections.append(detection)
        
        return all_detections

# SSD検出器のテスト
ssd_detector = SSDDetector()

# シミュレーションの特徴マップを生成
feature_maps = []
for size in [38, 19, 10, 5, 3, 1]:
    feature_maps.append(np.random.rand(size, size, 6 * (5 + 20)))

# 検出の実行
detections = ssd_detector.detect(feature_maps)
print(f"SSD検出結果: {len(detections)}個の物体を検出")

print("\nサンプル検出結果（最初の5件）:")
for i, det in enumerate(detections[:5]):
    print(f"物体{i+1}: 位置({det[0]:.2f}, {det[1]:.2f}), サイズ({det[2]:.2f}×{det[3]:.2f}),")
    print(f"          信頼度={det[4]:.2f}, クラスID={det[5]}")

## 2.7 Exercise 6: 可視化関数の実装

In [ ]:
def visualize_detections(image_size, detections, title="検出結果"):
    """検出結果を可視化
    
    Args:
        image_size: 画像サイズ [width, height]
        detections: 検出結果のリスト
        title: グラフのタイトル
    """
    plt.figure(figsize=(10, 8))
    
    # 背景を描画
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.gca().invert_yaxis()
    
    # ランダムな色の生成
    colors = plt.cm.rainbow(np.linspace(0, 1, len(detections)))
    
    # 各検出を描画
    for i, det in enumerate(detections):
        x, y, w, h = det[0], det[1], det[2], det[3]
        confidence = det[4]
        class_id = det[5]
        
        # 矩形の座標を計算
        x_min = x - w/2
        y_min = y - h/2
        x_max = x + w/2
        y_max = y + h/2
        
        # 矩形を描画
        rect = Rectangle((x_min, y_min), w, h, 
                        fill=False, 
                        edgecolor=colors[i],
                        linewidth=2,
                        label=f"Class {class_id}: {confidence:.2f}")
        plt.gca().add_patch(rect)
        
        # ラベルを描画
        plt.text(x_min, y_min - 0.05, f"C{class_id}", 
                color=colors[i], fontsize=10, weight='bold')
    
    plt.title(title)
    plt.xlabel('X (正規化座標)')
    plt.ylabel('Y (正規化座標)')
    plt.grid(True, alpha=0.3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# YOLO検出結果の可視化
print("YOLO検出結果の可視化:")
visualize_detections([1, 1], detections, "YOLO検出結果")

## 2.8 Exercise 7: 検出性能の比較

In [ ]:
def simulate_ground_truth(num_objects=5):
    """Ground Truthをシミュレート
    
    Args:
        num_objects: 物体の数
    
    Returns:
        Ground Truthのリスト
    """
    ground_truth = []
    
    for i in range(num_objects):
        # ランダムな位置とサイズ
        x = random.uniform(0.2, 0.8)
        y = random.uniform(0.2, 0.8)
        w = random.uniform(0.1, 0.3)
        h = random.uniform(0.1, 0.3)
        
        # クラスID（0-4の5クラス）
        class_id = i % 5
        
        ground_truth.append([x, y, w, h, 1.0, class_id])
    
    return ground_truth

def calculate_precision_recall(detections, ground_truth, iou_threshold=0.5):
    """PrecisionとRecallを計算
    
    Args:
        detections: 検出結果
        ground_truth: 正解データ
        iou_threshold: IoUの閾値
    
    Returns:
        precision, recall
    """
    true_positives = 0
    false_positives = 0
    
    # 各検出に対してGround Truthと比較
    used_gt = [False] * len(ground_truth)
    
    for det in detections:
        best_iou = 0
        best_gt_idx = -1
        
        # Ground TruthとのIoUを計算
        for i, gt in enumerate(ground_truth):
            if not used_gt[i] and det[5] == gt[5]:  # 同じクラス
                iou = calculate_iou(
                    [det[0]-det[2]/2, det[1]-det[3]/2, det[0]+det[2]/2, det[1]+det[3]/2],
                    [gt[0]-gt[2]/2, gt[1]-gt[3]/2, gt[0]+gt[2]/2, gt[1]+gt[3]/2]
                )
                
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = i
        
        if best_iou > iou_threshold:
            true_positives += 1
            used_gt[best_gt_idx] = True
        else:
            false_positives += 1
    
    # Recallの計算
    false_negatives = sum(1 for used in used_gt if not used)
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    
    return precision, recall

# Ground Truthを生成
ground_truth = simulate_ground_truth(num_objects=5)
print(f"Ground Truth: {len(ground_truth)}個の物体")

# YOLOの性能を測定
yolo_precision, yolo_recall = calculate_precision_recall(detections, ground_truth)

print(f"\nYOLOの性能:")
print(f"Precision: {yolo_precision:.3f}")
print(f"Recall: {yolo_recall:.3f}")
print(f"F1 Score: {2 * yolo_precision * yolo_recall / (yolo_precision + yolo_recall):.3f}")

## 2.9 Exercise 8: データ拡張

物体検出ではデータ拡張が重要です。

In [ ]:
class DataAugmentation:
    """データ拡張クラス"""
    
    @staticmethod
    def flip_horizontal(image, bboxes):
        """水平反転
        
        Args:
            image: 画像データ（ここではダミー）
            bboxes: バウンディングボックス [x, y, w, h, confidence, class_id]
        
        Returns:
            反転後のバウンディングボックス
        """
        flipped = []
        for bbox in bboxes:
            x, y, w, h, conf, cls = bbox
            # X座標を反転
            flipped.append([1 - x, y, w, h, conf, cls])
        return flipped
    
    @staticmethod
    def random_crop(image, bboxes, crop_ratio=0.8):
        """ランダムなクロップ
        
        Args:
            image: 画像データ
            bboxes: バウンディングボックス
            crop_ratio: クロップする割合

        Returns:
            クロップ後のバウンディングボックス
        """
        # クロップ領域を決定
        crop_size = 1.0 * crop_ratio
        crop_x = random.uniform(0, 1 - crop_size)
        crop_y = random.uniform(0, 1 - crop_size)
        
        cropped = []
        for bbox in bboxes:
            x, y, w, h, conf, cls = bbox
            
            # クロップ領域内にあるかチェック
            if (x - w/2 > crop_x and x + w/2 < crop_x + crop_size and
                y - h/2 > crop_y and y + h/2 < crop_y + crop_size):
                # クロップ領域内の相対座標に変換
                new_x = (x - crop_x) / crop_size
                new_y = (y - crop_y) / crop_size
                # サイズも調整
                new_w = w / crop_size
                new_h = h / crop_size
                
                cropped.append([new_x, new_y, new_w, new_h, conf, cls])
        
        return cropped

# データ拡張のテスト
detections = [
    [0.3, 0.5, 0.2, 0.3, 0.9, 0],
    [0.7, 0.6, 0.15, 0.2, 0.8, 1],
    [0.2, 0.3, 0.1, 0.1, 0.7, 2]
]

print("元の検出:")
for det in detections:
    print(f"位置({det[0]:.2f}, {det[1]:.2f}), サイズ({det[2]:.2f}×{det[3]:.2f})")

# 水平反転
flipped = DataAugmentation.flip_horizontal(None, detections)
print("\n水平反転後:")
for det in flipped:
    print(f"位置({det[0]:.2f}, {det[1]:.2f}), サイズ({det[2]:.2f}×{det[3]:.2f})")

# ランダムクロップ
cropped = DataAugmentation.random_crop(None, detections)
print(f"\nランダムクロップ後（{len(cropped)}個の物体が残りました）:")
for det in cropped:
    print(f"位置({det[0]:.2f}, {det[1]:.2f}), サイズ({det[2]:.2f}×{det[3]:.2f})")

## 2.10 Challenge: 物体検出システムの構築

これまで学んだ要素を使って、完全な物体検出システムを構築しましょう。

In [ ]:
class ObjectDetectionSystem:
    """完全な物体検出システム"""
    
    def __init__(self, detector_type='yolo'):
        self.detector_type = detector_type
        
        if detector_type == 'yolo':
            self.detector = YOLODetector()
        elif detector_type == 'ssd':
            self.detector = SSDDetector()
        
        self.confidence_threshold = 0.5
        self.iou_threshold = 0.5
    
    def detect(self, image_features, apply_augmentation=False):
        """物体検出を実行
        
        Args:
            image_features: 画像の特徴マップ
            apply_augmentation: データ拡張を適用するか
        
        Returns:
            検出結果
        """
        # 検出を実行
        if self.detector_type == 'yolo':
            predictions = self.detector.forward(image_features)
            detections = self.detector.filter_detections(predictions)
        else:  # SSD
            detections = self.detector.detect(image_features)
        
        # データ拡張を適用
        if apply_augmentation:
            detections = DataAugmentation.flip_horizontal(None, detections)
            
        return detections
    
    def evaluate(self, test_images, ground_truths):
        """システムの性能を評価
        
        Args:
            test_images: テスト画像のリスト
            ground_truths: 各画像のGround Truth
        
        Returns:
            評価結果
        """
        total_precision = 0
        total_recall = 0
        total_f1 = 0
        
        for i, (image, gt) in enumerate(zip(test_images, ground_truths)):
            # 検出を実行
            detections = self.detect(image)
            
            # 性能を計算
            precision, recall = calculate_precision_recall(detections, gt, self.iou_threshold)
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            
            total_precision += precision
            total_recall += recall
            total_f1 += f1
            
            # 進捗表示
            if (i + 1) % 10 == 0:
                print(f"処理中: {i+1}/{len(test_images)} images, 平均F1: {total_f1/(i+1):.3f}")
        
        # 平均を計算
        avg_precision = total_precision / len(test_images)
        avg_recall = total_recall / len(test_images)
        avg_f1 = total_f1 / len(test_images)
        
        return {
            'precision': avg_precision,
            'recall': avg_recall,
            'f1': avg_f1
        }
    
    def set_thresholds(self, confidence_threshold, iou_threshold):
        """閾値を設定
        
        Args:
            confidence_threshold: 信頼度の閾値
            iou_threshold: NMSのIoU閾値
        """
        self.confidence_threshold = confidence_threshold
        self.iou_threshold = iou_threshold
        
        if self.detector_type == 'yolo':
            self.detector.confidence_threshold = confidence_threshold
            self.detector.iou_threshold = iou_threshold

# システムのテスト
print("物体検出システムのテスト:")

# YOLOシステムの作成と評価
yolo_system = ObjectDetectionSystem(detector_type='yolo')
yolo_system.set_thresholds(0.6, 0.5)

# シミュレーションデータを作成
test_images = [np.random.rand(7, 7, 2 * (5 + 20)) for _ in range(20)]
ground_truths = [simulate_ground_truth(num_objects=3) for _ in range(20)]

# 評価実行（速度のためサブセットを使用）
evaluation = yolo_system.evaluate(test_images[:5], ground_truths[:5])

print("\nYOLOシステムの評価結果:")
print(f"平均Precision: {evaluation['precision']:.3f}")
print(f"平均Recall: {evaluation['recall']:.3f}")
print(f"平均F1 Score: {evaluation['f1']:.3f}")

# SSDシステムの比較
print("\nSSDシステムの比較:")
ssd_system = ObjectDetectionSystem(detector_type='ssd')
ssd_system.set_thresholds(0.6, 0.5)

# シミュレーションデータを調整
ssd_test_images = []
for size in [38, 19, 10, 5, 3, 1]:
    ssd_test_images.append(np.random.rand(size, size, 6 * (5 + 20)))

# SSDの評価
ssd_detections = ssd_system.detect(ssd_test_images)
print(f"SSD検出数: {len(ssd_detections)}")

print("\nシステム構築完了！")

---

# Self-Check (理解度確認)

本日の学習内容を確認しましょう：

## 基礎知識
- [ ] 物体検出の概念（位置+クラス検出）を理解した
- [ ] Two-stageとOne-stage検出器の違いを理解した
- [ ] YOLOとSSDの基本原理を理解した

## アルゴリズム
- [ ] **Anchor boxesの概念と生成方法を理解した**
- [ ] **NMSのアルゴリズムを理解・実装した**
- [ ] **IoUの計算方法を理解した**

## 実装力
- [ ] Simple Detectorを実装した
- [ ] YOLO Detectorを実装した
- [ ] SSD Detectorを実装した
- [ ] NMSを実装した
- [ ] 検出結果を可視化した

## 応用
- [ ] データ拡張の概念を理解した
- [ ] 評価指標（Precision, Recall, F1）を理解した
- [ ] 完全な物体検出システムを構築した

---

**お疲れ様でした！** Day 27の学習はこれで終了です。

次回（Day 28）からは3つの最終プロジェクトを進めます：
1. 画像フィルタアプリケーション
2. 画像分類アプリケーション
3. 総合アプリケーション

復習課題：
1. このNotebookのコードをすべて実行し、結果を確認する
2. IoUの計算を手動で行ってみる
3. NMSを自分の言葉で説明できるように整理する
4. YOLOとSSDの長所・短所を表にまとめる